# Reader & Writer

### Objectives
1. Read from CSV files
2. Read from JSON files
3. Write DataFrame to files
4. Write DataFrame to tables
5. Write DataFrame to a Delta table

### Methods
- [DataFrameReader](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.html): `csv`, `json`, `option`, `schema`
- [DataFrameWriter](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.html): `mode`, `option`, `parquet`, `format`, `saveAsTable`
- [StructType](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/types.html): `toDDL`


### Read from CSV with schema and options

In [ ]:
import pandas as pd

users_data = [
    {"user_id": "u001", "user_first_touch_timestamp": 1650000000000, "email": "user1@example.com"},
    {"user_id": "u002", "user_first_touch_timestamp": 1650000500000, "email": "user2@example.com"}
]
pdf = pd.DataFrame(users_data)
csv_path = "/dbfs/tmp/users_data.csv"
pdf.to_csv(csv_path, sep="\t", index=False)

from pyspark.sql.types import LongType, StringType, StructType, StructField

user_defined_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("user_first_touch_timestamp", LongType(), True),
    StructField("email", StringType(), True)
])

users_df = (spark.read
           .option("sep", "\t")
           .option("header", True)
           .schema(user_defined_schema)
           .csv(csv_path)
          )

users_df.printSchema()
display(users_df)

### Read from JSON with StructType schema

In [ ]:
events_data = [
    {
        "device": "mobile",
        "ecommerce": {"purchaseRevenue": 123.45, "total_item_quantity": 2, "unique_items": 2},
        "event_name": "purchase",
        "event_previous_timestamp": 1650000000000,
        "event_timestamp": 1650000600000,
        "geo": {"city": "Raleigh", "state": "NC"},
        "items": [{"coupon": None, "item_id": "i001", "item_name": "Item A", "item_revenue_in_usd": 123.45, "price_in_usd": 123.45, "quantity": 2}],
        "traffic_source": "adwords",
        "user_first_touch_timestamp": 1650000000000,
        "user_id": "u001"
    }
]

json_path = "/dbfs/tmp/events_data.json"
pd.DataFrame(events_data).to_json(json_path, orient="records", lines=True)

from pyspark.sql.types import ArrayType, DoubleType, LongType, StructType, StructField

schema = StructType([
    StructField("device", StringType(), True),
    StructField("ecommerce", StructType([
        StructField("purchaseRevenue", DoubleType(), True),
        StructField("total_item_quantity", LongType(), True),
        StructField("unique_items", LongType(), True)
    ])),
    StructField("event_name", StringType(), True),
    StructField("event_previous_timestamp", LongType(), True),
    StructField("event_timestamp", LongType(), True),
    StructField("geo", StructType([
        StructField("city", StringType(), True),
        StructField("state", StringType(), True)
    ])),
    StructField("items", ArrayType(StructType([
        StructField("coupon", StringType(), True),
        StructField("item_id", StringType(), True),
        StructField("item_name", StringType(), True),
        StructField("item_revenue_in_usd", DoubleType(), True),
        StructField("price_in_usd", DoubleType(), True),
        StructField("quantity", LongType(), True)
    ]))),
    StructField("traffic_source", StringType(), True),
    StructField("user_first_touch_timestamp", LongType(), True),
    StructField("user_id", StringType(), True)
])

events_df = spark.read.schema(schema).json(json_path)
events_df.printSchema()
display(events_df)

### Write users_df to Parquet

In [ ]:
parquet_path = "/dbfs/tmp/output_users.parquet"

(users_df.write
 .option("compression", "snappy")
 .mode("overwrite")
 .parquet(parquet_path))

display(dbutils.fs.ls(parquet_path))

### Save events_df as a table

In [ ]:
events_df.write.mode("overwrite").saveAsTable("events_table")

### Write events_df to Delta Table

In [ ]:
delta_path = "/dbfs/tmp/delta/events_output"

(events_df.write
 .format("delta")
 .mode("overwrite")
 .save(delta_path))